In [2]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
import json

from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, LSTM, Embedding, SpatialDropout1D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import BatchNormalization, Activation, ZeroPadding2D

Using TensorFlow backend.


In [3]:
df_idf = pd.read_csv('amazon/reviews.csv')
df_dataset = pd.read_json('clothing_dataset/renttherunway_final_data.json', lines = True)

In [4]:
def pre_process(text):
    # to lowercase
    text=text.lower()
    
    # remove tags
    text = re.sub("&lt;/?.*?&gt;", "&lt;&gt; ", text)
    
    # remove special characters and digits
    text = re.sub("(\\d|\\W)+", " ", text)
    
    return text

def get_stop_words(stop_file_path):
    with open(stop_file_path, 'r', encoding='utf-8') as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return frozenset(stop_set)

In [5]:
df_idf['text'] = df_idf['title'] + " " + df_idf['body']
df_idf['text'] = df_idf['text'].apply(lambda x: pre_process(str(x)))

df_dataset['text'] = df_dataset['review_summary'] + " " + df_dataset['review_text']
df_dataset['text'] = df_dataset['text'].apply(lambda x: pre_process(str(x)))

sub_dataset = df_dataset[['text', 'rating']]

In [6]:
all_data = df_idf['text'].append(sub_dataset['text'])
all_rating = df_idf['rating'].append(sub_dataset['rating'])

In [7]:
y = all_rating
y[:len(df_idf)] = y[:len(df_idf)].apply(lambda x: 1 if x > 3.5 else 0)#y.apply(lambda x: 1 if x > 3.5 else 0) 
y[len(df_idf):] = y[len(df_idf):].apply(lambda x: 1 if x > 5 else 0)#y.apply(lambda x: 1 if x > 3.5 else 0) 
y = y.to_numpy()

In [8]:
# Change this to use both datasets
#X_train, X_test, y_train, y_test = train_test_split(all_data, y, test_size=0.1, random_state=37)
# X_train = df_dataset['text']
# y_train = y[len(df_idf):]

# X_test = df_idf['text'].to_numpy()
# y_test = y[:len(df_idf)]

# print('# Train data samples:', X_train.shape)
# print('# Test data samples:', X_test.shape)

# print('Sample train', X_train[0])
# print('\nSample test', X_test[0])
# print(X_train.shape, y_train.shape)
# print(X_test.shape, y_test.shape)
# assert X_train.shape[0] == y_train.shape[0]
# assert X_test.shape[0] == y_test.shape[0]

In [9]:
MAX_LEN = 5260
GLOVE_DIM = 300
NB_WORDS = 49781

In [10]:
from keras.preprocessing.text import Tokenizer

tk = Tokenizer(num_words=NB_WORDS,
               filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
               split=" ")
tk.fit_on_texts(all_data)

all_data_seq = tk.texts_to_sequences(all_data)

In [11]:
X_train_seq = all_data_seq[len(df_idf):]
X_test_seq = all_data_seq[:len(df_idf)]

y_train = y[len(df_idf):]
y_test = y[:len(df_idf)]

In [12]:
seq_lengths = all_data.apply(lambda x: len(x.split(' ')))
print(seq_lengths.describe())

count    275359.000000
mean         63.517557
std          69.966379
min           1.000000
25%          26.000000
50%          50.000000
75%          82.000000
max        5260.000000
Name: text, dtype: float64


In [13]:
X_train_seq_trunc = pad_sequences(X_train_seq, maxlen=MAX_LEN)
X_test_seq_trunc = pad_sequences(X_test_seq, maxlen=MAX_LEN)

In [ ]:
# This is to split the X_test_seq_trunc in two normalize dataset for GANs
print(X_test_seq_trunc.shape)

positives = X_test_seq_trunc[0:1]
negatives = X_test_seq_trunc[0:1]

normalized_data = X_test_seq_trunc[0:1]

print(positives.shape)

for i in range(0, y_test.shape[0]):
    if y_test[i] == 1:
        positives = np.append(positives, X_test_seq_trunc[i:(i+1)], axis=0)
    else:
        negatives = np.append(negatives, X_test_seq_trunc[i:(i+1)], axis=0)
        
iter_ = min(positives.shape[0], negatives.shape[0])
for i in range(0, iter_):
    normalized_data = np.append(normalized_data, positives[i:(i+1)], axis=0)
    normalized_data = np.append(normalized_data, negatives[i:(i+1)], axis=0)
    
        
print(positives.shape, negatives.shape, normalized_data.shape)

# for i in range(0, 5000):
#     if(positives[0:1, i] != 0):
#         print(positives[0:1, i], X_test_seq_trunc[0:1, i], i)
            
    

(82815, 5260)
(1, 5260)


In [58]:
print(X_train_seq_trunc[10])  # Example of padded sequence
print(X_train_seq_trunc.shape)
print(X_test_seq_trunc.shape)
print(y_train.shape)
print(y_test.shape)

[  0   0   0 ...  61  14 316]
(192544, 5260)
(82815, 5260)
(192544,)
(82815,)


In [326]:
X_train_emb, X_valid_emb, y_train_emb, y_valid_emb = train_test_split(X_train_seq_trunc, y_train, test_size=0.1, random_state=37)

assert X_valid_emb.shape[0] == y_valid_emb.shape[0]
assert X_train_emb.shape[0] == y_train_emb.shape[0]

print('Shape of validation set:', X_valid_emb.shape)

Shape of validation set: (19255, 5260)


In [327]:
# This read the embeddings
# glove_file = 'glove.42B.' + str(GLOVE_DIM) + 'd.txt'
# emb_dict = {}
# glove = open(glove_file)
# for line in glove:
#     values = line.split()
#     word = values[0]
#     vector = np.asarray(values[1:], dtype='float32')
#     emb_dict[word] = vector
# glove.close()

In [328]:
airline_words = ['car', 'nice', 'flight', 'luggage']
for w in airline_words:
    if w in emb_dict.keys():
        print('Found the word {} in the dictionary'.format(w))
# print(emb_dict)

Found the word car in the dictionary
Found the word nice in the dictionary
Found the word flight in the dictionary
Found the word luggage in the dictionary


In [329]:
# Here we build a matrix that represent words and it corresponding emdg
emb_matrix = np.zeros((NB_WORDS, GLOVE_DIM))

for w, i in tk.word_index.items():
    if i < NB_WORDS:
        vect = emb_dict.get(w)
        if vect is not None:
            emb_matrix[i] = vect
    else:
        break

In [330]:
glove_model = Sequential()
glove_model.add(Embedding(NB_WORDS, GLOVE_DIM, input_length=MAX_LEN))
# glove_model.add(LSTM(GLOVE_DIM, return_sequences=True))
# glove_model.add(LSTM(GLOVE_DIM, return_sequences=True))
glove_model.add(Flatten())
glove_model.add(Dense(1, activation='sigmoid'))
glove_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 5260, 300)         14934300  
_________________________________________________________________
flatten_15 (Flatten)         (None, 1578000)           0         
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 1578001   
Total params: 16,512,301
Trainable params: 16,512,301
Non-trainable params: 0
_________________________________________________________________


In [331]:
glove_model.layers[0].set_weights([emb_matrix])
glove_model.layers[0].trainable = False

glove_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [332]:
history = glove_model.fit(X_train_emb
                       , y_train_emb
                       , epochs=5
                       , batch_size=32
                       , validation_data=(X_valid_emb, y_valid_emb)
                       , verbose=1)

Train on 173289 samples, validate on 19255 samples
Epoch 1/5
173289/173289 [==============================] - 570s 3ms/step - loss: 0.1126 - acc: 0.9748 - val_loss: 0.1106 - val_acc: 0.9775
Epoch 2/5
173289/173289 [==============================] - 2681s 15ms/step - loss: 0.0786 - acc: 0.9793 - val_loss: 0.1193 - val_acc: 0.9787
Epoch 3/5
173289/173289 [==============================] - 1324s 8ms/step - loss: 0.0669 - acc: 0.9813 - val_loss: 0.1276 - val_acc: 0.9747
Epoch 4/5
173289/173289 [==============================] - 566s 3ms/step - loss: 0.0582 - acc: 0.9837 - val_loss: 0.1401 - val_acc: 0.9671
Epoch 5/5
173289/173289 [==============================] - 1143s 7ms/step - loss: 0.0520 - acc: 0.9848 - val_loss: 0.1319 - val_acc: 0.9752


In [333]:
print(glove_model.predict(X_train_emb[0:1]))
print(X_train_emb[0].shape)

[[0.9999999]]
(5260,)


# Adversarial Neural Network

In this section we start building the GANs, this model takes the word embedding and generate new embeddings that are similar to the given ones. 

In [340]:
def build_generator(shape):
    img_shape = shape
    noise_shape = (100,)

    model = Sequential()

    model.add(Dense(256, input_shape=noise_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))

    model.summary()

    noise = Input(shape=noise_shape)
    img = model(noise)

    return Model(noise, img)

def build_discriminator(shape):

    img_shape = shape

    model = Sequential()

#     model.add(Flatten(input_shape=img_shape)) # is one dimension
    model.add(Dense(512, input_shape=img_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)

    return Model(img, validity)


In [341]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
def results(pred, actual):
    results = confusion_matrix(actual, pred)
    print('Confusion Matrix :')
    print(results)
    print ('Accuracy Score :',accuracy_score(actual, pred))
    print ('Report : ')
    print(classification_report(actual, pred))
    print()

In [342]:
img_rows = 1
img_cols = X_train_emb[0].shape
img_shape = (img_cols)

optimizer = Adam(0.0002, 0.5)

# Build and compile the discriminator
discriminator = build_discriminator(img_shape)
discriminator.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

# Build and compile the generator
generator = build_generator(img_shape)
generator.compile(loss='binary_crossentropy', optimizer=optimizer)

# The generator takes noise as input and generated imgs
z = Input(shape=(100,))
img = generator(z)

# For the combined model we will only train the generator
discriminator.trainable = False

# The valid takes generated images as input and determines validity
valid = discriminator(img)

# The combined model  (stacked generator and discriminator) takes
# noise as input => generates images => determines validity
combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_29 (Dense)             (None, 512)               2693632   
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_30 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_31 (Dense)             (None, 1)                 257       
Total params: 2,825,217
Trainable params: 2,825,217
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   


In [343]:
def train(epochs, data, batch_size=128):


        # Rescale -1 to 1
        X_train = data #(X_train.astype(np.float32) - 127.5) / 127.5
#         X_train = np.expand_dims(X_train, axis=3)

        half_batch = int(batch_size / 2)

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[1], half_batch)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (half_batch, 100))

            # Generate a half batch of new images
            gen_imgs = np.round(generator.predict(noise))

            # Train the discriminator
            d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, 100))

            # The generator wants the discriminator to label the generated samples
            # as valid (ones)
            valid_y = np.array([1] * batch_size)

            # Train the generator
            g_loss = combined.train_on_batch(noise, valid_y)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))



In [344]:
train(epochs=4000, data=normalized_data) # Generate data similar to the second dataset

0 [D loss: 7.046728, acc.: 5.47%] [G loss: 0.871567]
1 [D loss: 5.264689, acc.: 31.25%] [G loss: 0.668643]
2 [D loss: 2.291618, acc.: 47.66%] [G loss: 0.505634]
3 [D loss: 0.591841, acc.: 54.69%] [G loss: 0.438582]
4 [D loss: 0.715919, acc.: 51.56%] [G loss: 0.385361]
5 [D loss: 0.947541, acc.: 50.78%] [G loss: 0.352522]
6 [D loss: 0.846476, acc.: 54.69%] [G loss: 0.350906]
7 [D loss: 0.820618, acc.: 50.78%] [G loss: 0.292279]
8 [D loss: 0.819013, acc.: 53.12%] [G loss: 0.304908]
9 [D loss: 0.831732, acc.: 55.47%] [G loss: 0.298858]
10 [D loss: 1.168113, acc.: 51.56%] [G loss: 0.319684]
11 [D loss: 1.737201, acc.: 44.53%] [G loss: 0.275976]
12 [D loss: 1.398799, acc.: 49.22%] [G loss: 0.281492]
13 [D loss: 1.400958, acc.: 48.44%] [G loss: 0.274642]
14 [D loss: 1.210433, acc.: 48.44%] [G loss: 0.274221]
15 [D loss: 1.914791, acc.: 46.09%] [G loss: 0.254683]
16 [D loss: 1.041364, acc.: 53.12%] [G loss: 0.292025]
17 [D loss: 1.093817, acc.: 51.56%] [G loss: 0.244153]
18 [D loss: 1.440656,

149 [D loss: 1.037021, acc.: 67.97%] [G loss: 0.851063]
150 [D loss: 0.813021, acc.: 68.75%] [G loss: 0.912513]
151 [D loss: 0.639808, acc.: 69.53%] [G loss: 0.933183]
152 [D loss: 0.500529, acc.: 81.25%] [G loss: 0.946414]
153 [D loss: 0.904889, acc.: 67.97%] [G loss: 1.000593]
154 [D loss: 0.691594, acc.: 72.66%] [G loss: 0.864655]
155 [D loss: 0.760005, acc.: 67.19%] [G loss: 0.747582]
156 [D loss: 0.750624, acc.: 71.88%] [G loss: 0.925822]
157 [D loss: 0.573179, acc.: 75.00%] [G loss: 0.972421]
158 [D loss: 0.938702, acc.: 63.28%] [G loss: 0.911464]
159 [D loss: 0.570715, acc.: 72.66%] [G loss: 0.983700]
160 [D loss: 0.755350, acc.: 72.66%] [G loss: 0.913544]
161 [D loss: 0.944763, acc.: 67.19%] [G loss: 0.947576]
162 [D loss: 1.207706, acc.: 61.72%] [G loss: 0.720886]
163 [D loss: 0.915006, acc.: 64.06%] [G loss: 0.790592]
164 [D loss: 0.795564, acc.: 67.19%] [G loss: 0.850739]
165 [D loss: 0.916639, acc.: 65.62%] [G loss: 1.089210]
166 [D loss: 1.029608, acc.: 67.19%] [G loss: 1.

296 [D loss: 0.994799, acc.: 77.34%] [G loss: 1.535928]
297 [D loss: 1.159605, acc.: 78.12%] [G loss: 1.478069]
298 [D loss: 1.188054, acc.: 76.56%] [G loss: 1.467888]
299 [D loss: 0.993398, acc.: 84.38%] [G loss: 1.488040]
300 [D loss: 0.526023, acc.: 85.16%] [G loss: 1.472012]
301 [D loss: 1.054389, acc.: 82.03%] [G loss: 1.457672]
302 [D loss: 1.621373, acc.: 68.75%] [G loss: 1.394174]
303 [D loss: 1.425780, acc.: 77.34%] [G loss: 1.482499]
304 [D loss: 1.288463, acc.: 78.91%] [G loss: 1.545128]
305 [D loss: 1.023916, acc.: 82.81%] [G loss: 1.476797]
306 [D loss: 1.106613, acc.: 82.81%] [G loss: 1.497591]
307 [D loss: 0.888516, acc.: 82.03%] [G loss: 1.440814]
308 [D loss: 0.766526, acc.: 82.81%] [G loss: 1.427128]
309 [D loss: 1.150405, acc.: 82.81%] [G loss: 1.473580]
310 [D loss: 1.647521, acc.: 79.69%] [G loss: 1.530922]
311 [D loss: 1.114856, acc.: 83.59%] [G loss: 1.465862]
312 [D loss: 1.523160, acc.: 76.56%] [G loss: 1.515203]
313 [D loss: 1.369241, acc.: 82.81%] [G loss: 1.

443 [D loss: 1.121637, acc.: 83.59%] [G loss: 1.953333]
444 [D loss: 1.555476, acc.: 85.16%] [G loss: 1.649715]
445 [D loss: 1.502000, acc.: 79.69%] [G loss: 1.744407]
446 [D loss: 1.439832, acc.: 85.16%] [G loss: 1.913359]
447 [D loss: 2.327365, acc.: 77.34%] [G loss: 1.733206]
448 [D loss: 1.179938, acc.: 89.06%] [G loss: 1.686034]
449 [D loss: 1.600764, acc.: 80.47%] [G loss: 1.757928]
450 [D loss: 1.885378, acc.: 79.69%] [G loss: 1.873641]
451 [D loss: 1.053177, acc.: 85.16%] [G loss: 1.852402]
452 [D loss: 1.027224, acc.: 89.84%] [G loss: 1.715515]
453 [D loss: 1.831561, acc.: 81.25%] [G loss: 1.687727]
454 [D loss: 1.070896, acc.: 84.38%] [G loss: 1.576992]
455 [D loss: 1.841679, acc.: 81.25%] [G loss: 1.647445]
456 [D loss: 1.544781, acc.: 82.81%] [G loss: 1.773588]
457 [D loss: 1.415264, acc.: 85.16%] [G loss: 1.717303]
458 [D loss: 1.062562, acc.: 85.94%] [G loss: 1.824416]
459 [D loss: 1.684075, acc.: 86.72%] [G loss: 1.761310]
460 [D loss: 1.780554, acc.: 84.38%] [G loss: 1.

590 [D loss: 0.646942, acc.: 79.69%] [G loss: 1.590503]
591 [D loss: 0.611926, acc.: 76.56%] [G loss: 1.456531]
592 [D loss: 0.574012, acc.: 78.91%] [G loss: 1.604154]
593 [D loss: 0.567382, acc.: 73.44%] [G loss: 1.688564]
594 [D loss: 0.921788, acc.: 75.00%] [G loss: 2.038002]
595 [D loss: 0.319896, acc.: 84.38%] [G loss: 2.129745]
596 [D loss: 0.332532, acc.: 89.06%] [G loss: 1.718575]
597 [D loss: 0.346247, acc.: 82.03%] [G loss: 1.488289]
598 [D loss: 0.673642, acc.: 78.12%] [G loss: 1.476811]
599 [D loss: 0.402523, acc.: 81.25%] [G loss: 1.546272]
600 [D loss: 0.441125, acc.: 78.12%] [G loss: 1.493518]
601 [D loss: 0.311801, acc.: 85.16%] [G loss: 1.583756]
602 [D loss: 0.668707, acc.: 78.12%] [G loss: 1.630574]
603 [D loss: 0.603634, acc.: 78.91%] [G loss: 1.770874]
604 [D loss: 0.552058, acc.: 82.81%] [G loss: 1.678870]
605 [D loss: 0.921376, acc.: 75.00%] [G loss: 2.003044]
606 [D loss: 0.442968, acc.: 83.59%] [G loss: 1.985069]
607 [D loss: 0.790567, acc.: 81.25%] [G loss: 1.

...

3333 [D loss: 0.045289, acc.: 100.00%] [G loss: 2.509589]
3334 [D loss: 0.041999, acc.: 100.00%] [G loss: 2.343612]
3335 [D loss: 0.038314, acc.: 100.00%] [G loss: 2.546252]
3336 [D loss: 0.041581, acc.: 99.22%] [G loss: 2.264537]
3337 [D loss: 0.306085, acc.: 98.44%] [G loss: 2.704380]
3338 [D loss: 0.145076, acc.: 99.22%] [G loss: 2.358148]
3339 [D loss: 0.032540, acc.: 100.00%] [G loss: 2.174837]
3340 [D loss: 0.157837, acc.: 99.22%] [G loss: 2.453185]
3341 [D loss: 0.404315, acc.: 97.66%] [G loss: 2.298076]
3342 [D loss: 0.038665, acc.: 99.22%] [G loss: 2.246801]
3343 [D loss: 0.161740, acc.: 98.44%] [G loss: 2.140712]
3344 [D loss: 0.030360, acc.: 100.00%] [G loss: 1.724710]
3345 [D loss: 0.171112, acc.: 97.66%] [G loss: 2.087926]
3346 [D loss: 0.147168, acc.: 99.22%] [G loss: 1.780881]
3347 [D loss: 0.018577, acc.: 100.00%] [G loss: 1.998798]
3348 [D loss: 0.288678, acc.: 97.66%] [G loss: 1.824612]
3349 [D loss: 0.020631, acc.: 100.00%] [G loss: 1.696969]
3350 [D loss: 0.017393, 

3476 [D loss: 0.003631, acc.: 100.00%] [G loss: 3.181812]
3477 [D loss: 0.131536, acc.: 99.22%] [G loss: 3.406800]
3478 [D loss: 0.002325, acc.: 100.00%] [G loss: 3.168016]
3479 [D loss: 0.131896, acc.: 99.22%] [G loss: 2.884522]
3480 [D loss: 0.131131, acc.: 99.22%] [G loss: 2.632545]
3481 [D loss: 0.132290, acc.: 99.22%] [G loss: 2.527045]
3482 [D loss: 0.005981, acc.: 100.00%] [G loss: 2.671059]
3483 [D loss: 0.131428, acc.: 99.22%] [G loss: 2.451877]
3484 [D loss: 0.002711, acc.: 100.00%] [G loss: 2.672624]
3485 [D loss: 0.004053, acc.: 100.00%] [G loss: 2.601728]
3486 [D loss: 0.381074, acc.: 97.66%] [G loss: 2.845077]
3487 [D loss: 0.129991, acc.: 99.22%] [G loss: 3.152950]
3488 [D loss: 0.131915, acc.: 99.22%] [G loss: 3.057860]
3489 [D loss: 0.257405, acc.: 98.44%] [G loss: 2.703248]
3490 [D loss: 0.005480, acc.: 100.00%] [G loss: 2.594514]
3491 [D loss: 0.258540, acc.: 98.44%] [G loss: 2.457742]
3492 [D loss: 0.635865, acc.: 96.09%] [G loss: 2.475234]
3493 [D loss: 0.132656, a

3619 [D loss: 0.005055, acc.: 100.00%] [G loss: 2.669683]
3620 [D loss: 0.002939, acc.: 100.00%] [G loss: 3.295289]
3621 [D loss: 0.003341, acc.: 100.00%] [G loss: 3.095693]
3622 [D loss: 0.003238, acc.: 100.00%] [G loss: 3.020647]
3623 [D loss: 0.004264, acc.: 100.00%] [G loss: 2.758238]
3624 [D loss: 0.131044, acc.: 99.22%] [G loss: 2.558695]
3625 [D loss: 0.004361, acc.: 100.00%] [G loss: 2.587635]
3626 [D loss: 0.004946, acc.: 100.00%] [G loss: 2.736785]
3627 [D loss: 0.130772, acc.: 99.22%] [G loss: 2.702186]
3628 [D loss: 0.004013, acc.: 100.00%] [G loss: 2.901571]
3629 [D loss: 0.004612, acc.: 100.00%] [G loss: 2.838457]
3630 [D loss: 0.015888, acc.: 99.22%] [G loss: 3.129494]
3631 [D loss: 0.256441, acc.: 98.44%] [G loss: 2.908387]
3632 [D loss: 0.129028, acc.: 99.22%] [G loss: 3.213625]
3633 [D loss: 0.005110, acc.: 100.00%] [G loss: 2.933588]
3634 [D loss: 0.003862, acc.: 100.00%] [G loss: 2.976747]
3635 [D loss: 0.004104, acc.: 100.00%] [G loss: 2.907045]
3636 [D loss: 0.003

3762 [D loss: 0.129352, acc.: 99.22%] [G loss: 3.013406]
3763 [D loss: 0.024065, acc.: 99.22%] [G loss: 2.173384]
3764 [D loss: 0.017950, acc.: 100.00%] [G loss: 2.743493]
3765 [D loss: 0.038515, acc.: 98.44%] [G loss: 4.637526]
3766 [D loss: 0.435385, acc.: 85.16%] [G loss: 7.671311]
3767 [D loss: 2.748100, acc.: 81.25%] [G loss: 9.864925]
3768 [D loss: 2.195925, acc.: 85.16%] [G loss: 12.257677]
3769 [D loss: 0.710827, acc.: 93.75%] [G loss: 14.422079]
3770 [D loss: 0.378230, acc.: 96.88%] [G loss: 14.822724]
3771 [D loss: 0.189960, acc.: 96.88%] [G loss: 15.676813]
3772 [D loss: 0.086569, acc.: 98.44%] [G loss: 16.082630]
3773 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.110493]
3774 [D loss: 0.125923, acc.: 99.22%] [G loss: 16.009245]
3775 [D loss: 0.125923, acc.: 99.22%] [G loss: 16.047432]
3776 [D loss: 0.000000, acc.: 100.00%] [G loss: 15.636271]
3777 [D loss: 0.000174, acc.: 100.00%] [G loss: 15.418091]
3778 [D loss: 0.125923, acc.: 99.22%] [G loss: 15.178413]
3779 [D loss: 0.

3905 [D loss: 0.129178, acc.: 99.22%] [G loss: 3.313347]
3906 [D loss: 0.002955, acc.: 100.00%] [G loss: 3.272816]
3907 [D loss: 0.254195, acc.: 98.44%] [G loss: 3.155061]
3908 [D loss: 0.002360, acc.: 100.00%] [G loss: 3.206047]
3909 [D loss: 0.002182, acc.: 100.00%] [G loss: 3.585164]
3910 [D loss: 0.255112, acc.: 98.44%] [G loss: 3.692319]
3911 [D loss: 0.003590, acc.: 100.00%] [G loss: 3.481729]
3912 [D loss: 0.129042, acc.: 99.22%] [G loss: 3.134404]
3913 [D loss: 0.003955, acc.: 100.00%] [G loss: 3.017752]
3914 [D loss: 0.255825, acc.: 98.44%] [G loss: 3.223722]
3915 [D loss: 0.003632, acc.: 100.00%] [G loss: 3.198283]
3916 [D loss: 0.128659, acc.: 99.22%] [G loss: 3.225466]
3917 [D loss: 0.128493, acc.: 99.22%] [G loss: 3.318871]
3918 [D loss: 0.380769, acc.: 97.66%] [G loss: 3.189687]
3919 [D loss: 0.003241, acc.: 100.00%] [G loss: 3.302076]
3920 [D loss: 0.002501, acc.: 100.00%] [G loss: 3.559956]
3921 [D loss: 0.380996, acc.: 97.66%] [G loss: 3.368517]
3922 [D loss: 0.128405,

# Model Evaluation

Second model trained on the real data

In [345]:
print(X_test_seq_trunc.shape, y_test.shape)

(82815, 5260) (82815,)


In [346]:
X_train_emb2, X_valid_emb2, y_train_emb2, y_valid_emb2 = train_test_split(X_test_seq_trunc, y_test, test_size=0.1, random_state=37)

assert X_valid_emb2.shape[0] == y_valid_emb2.shape[0]
assert X_train_emb2.shape[0] == y_train_emb2.shape[0]

print('Shape of validation set:',X_valid_emb2.shape)

Shape of validation set: (8282, 5260)


In [347]:
glove_model2 = Sequential()
glove_model2.add(Embedding(NB_WORDS, GLOVE_DIM, input_length=MAX_LEN))
# glove_model2.add(LSTM(GLOVE_DIM, return_sequences=True))
# glove_model2.add(LSTM(GLOVE_DIM, return_sequences=True))
glove_model2.add(Flatten())
glove_model2.add(Dense(1, activation='sigmoid'))
glove_model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, 5260, 300)         14934300  
_________________________________________________________________
flatten_16 (Flatten)         (None, 1578000)           0         
_________________________________________________________________
dense_36 (Dense)             (None, 1)                 1578001   
Total params: 16,512,301
Trainable params: 16,512,301
Non-trainable params: 0
_________________________________________________________________


In [348]:
glove_model2.layers[0].set_weights([emb_matrix])
glove_model2.layers[0].trainable = False

glove_model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [349]:
history2 = glove_model2.fit(X_train_emb2
                       , y_train_emb2
                       , epochs=5
                       , batch_size=32
                       , validation_data=(X_valid_emb2, y_valid_emb2)
                       , verbose=1)

Train on 74533 samples, validate on 8282 samples
Epoch 1/5
74533/74533 [==============================] - 339s 5ms/step - loss: 0.4503 - acc: 0.8394 - val_loss: 0.4638 - val_acc: 0.8604
Epoch 2/5
74533/74533 [==============================] - 394s 5ms/step - loss: 0.3001 - acc: 0.8967 - val_loss: 0.4647 - val_acc: 0.8662
Epoch 3/5
74533/74533 [==============================] - 712s 10ms/step - loss: 0.2538 - acc: 0.9116 - val_loss: 0.4629 - val_acc: 0.8694
Epoch 4/5
74533/74533 [==============================] - 613s 8ms/step - loss: 0.2288 - acc: 0.9208 - val_loss: 0.4886 - val_acc: 0.8743
Epoch 5/5
74533/74533 [==============================] - 303s 4ms/step - loss: 0.2023 - acc: 0.9291 - val_loss: 0.5952 - val_acc: 0.8624


In [350]:
# print(glove_model2.predict(X_train_emb[0:1]))
# print(X_train_emb[0].shape)

Third model trained on a generated dataset

In [351]:
# Generating data and labels
gen = 100000
noise = np.random.normal(0, 1, (gen, 100))
gen_samp = np.absolute((generator.predict(noise)))
prediction = glove_model.predict((gen_samp))
prediction = np.round(prediction)

In [352]:

# print(np.round(prediction[0:100]))
# print(np.sum(np.round(glove_model.predict(X_train_emb[0:100]))))
# print(np.sum(y_train[0:100]))


In [353]:
X_train_emb3, X_valid_emb3, y_train_emb3, y_valid_emb3 = train_test_split(gen_samp, prediction, test_size=0.3, random_state=37)

assert X_valid_emb3.shape[0] == y_valid_emb3.shape[0]
assert X_train_emb3.shape[0] == y_train_emb3.shape[0]

print('Shape of validation set:',X_valid_emb3.shape)

Shape of validation set: (30000, 5260)


In [354]:
glove_model3 = Sequential()
glove_model3.add(Embedding(NB_WORDS, GLOVE_DIM, input_length=MAX_LEN))
# glove_model3.add(LSTM(GLOVE_DIM, return_sequences=True))
# glove_model3.add(LSTM(GLOVE_DIM, return_sequences=True))
glove_model3.add(Flatten())
glove_model3.add(Dense(1, activation='sigmoid'))
glove_model3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_17 (Embedding)     (None, 5260, 300)         14934300  
_________________________________________________________________
flatten_17 (Flatten)         (None, 1578000)           0         
_________________________________________________________________
dense_37 (Dense)             (None, 1)                 1578001   
Total params: 16,512,301
Trainable params: 16,512,301
Non-trainable params: 0
_________________________________________________________________


In [355]:
glove_model3.layers[0].set_weights([emb_matrix])
glove_model3.layers[0].trainable = False

glove_model3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [356]:
history3 = glove_model3.fit(X_train_emb3
                       , y_train_emb3
                       , epochs=5
                       , batch_size=32
                       , validation_data=(X_valid_emb3, y_valid_emb3)
                       , verbose=1)

Train on 70000 samples, validate on 30000 samples
Epoch 1/5
70000/70000 [==============================] - 412s 6ms/step - loss: 0.1680 - acc: 0.9944 - val_loss: 0.0778 - val_acc: 0.9992
Epoch 2/5
70000/70000 [==============================] - 681s 10ms/step - loss: 0.0477 - acc: 0.9987 - val_loss: 0.0289 - val_acc: 0.9983
Epoch 3/5
70000/70000 [==============================] - 438s 6ms/step - loss: 0.0186 - acc: 0.9990 - val_loss: 0.0139 - val_acc: 0.9987
Epoch 4/5
70000/70000 [==============================] - 669s 10ms/step - loss: 0.0095 - acc: 0.9989 - val_loss: 0.0062 - val_acc: 0.9992
Epoch 5/5
70000/70000 [==============================] - 456s 7ms/step - loss: 0.0046 - acc: 0.9994 - val_loss: 0.0048 - val_acc: 0.9990


In this area I am going to compare their results

In [357]:
# The model trained on the generated dataset over the real dataset
actual = y_valid_emb3
pred = np.round(glove_model2.predict(X_valid_emb3))
pred2 = np.round(glove_model3.predict(X_valid_emb3))

In [358]:
results(pred, actual) # trained on original over real
results(pred2, actual) # generated

Confusion Matrix :
[[ 3801  2687]
 [ 3993 19519]]
Accuracy Score : 0.7773333333333333
Report : 
              precision    recall  f1-score   support

         0.0       0.49      0.59      0.53      6488
         1.0       0.88      0.83      0.85     23512

   micro avg       0.78      0.78      0.78     30000
   macro avg       0.68      0.71      0.69     30000
weighted avg       0.79      0.78      0.78     30000


Confusion Matrix :
[[ 6466    22]
 [    9 23503]]
Accuracy Score : 0.9989666666666667
Report : 
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      6488
         1.0       1.00      1.00      1.00     23512

   micro avg       1.00      1.00      1.00     30000
   macro avg       1.00      1.00      1.00     30000
weighted avg       1.00      1.00      1.00     30000




In [359]:
# The model trained on the generated dataset over the real dataset
actual = y_valid_emb2
pred = np.round(glove_model2.predict(X_valid_emb2))
pred2 = np.round(glove_model3.predict(X_valid_emb2))

In [360]:
results(pred, actual) # trained on original over real
results(pred2, actual) # generated

Confusion Matrix :
[[1943  765]
 [ 375 5199]]
Accuracy Score : 0.8623520888674233
Report : 
              precision    recall  f1-score   support

         0.0       0.84      0.72      0.77      2708
         1.0       0.87      0.93      0.90      5574

   micro avg       0.86      0.86      0.86      8282
   macro avg       0.85      0.83      0.84      8282
weighted avg       0.86      0.86      0.86      8282


Confusion Matrix :
[[   1 2707]
 [   0 5574]]
Accuracy Score : 0.6731465829509781
Report : 
              precision    recall  f1-score   support

         0.0       1.00      0.00      0.00      2708
         1.0       0.67      1.00      0.80      5574

   micro avg       0.67      0.67      0.67      8282
   macro avg       0.84      0.50      0.40      8282
weighted avg       0.78      0.67      0.54      8282


